### Import libraries

In [1]:
import polars as pl
import os
from deltalake import DeltaTable
from lakefs import Repository
from lakefs.client import Client
from dotenv import load_dotenv

load_dotenv()

True

### Configure lakefs client, storage options

In [2]:
lakefs_endpoint = "http://127.0.0.1:8000"

lakefs_client = Client(
    host=lakefs_endpoint,
    username=os.environ["LAKEFS_ACCESS_KEY"],
    password=os.environ["LAKEFS_SECRET_ACCESS_KEY"],
)
storage_options={
        "access_key_id": os.environ["LAKEFS_ACCESS_KEY"], 
        "secret_access_key": os.environ["LAKEFS_SECRET_ACCESS_KEY"], 
        "endpoint":lakefs_endpoint,
        "allow_http": "true",
        }

bronze_repo = Repository("bronze", client=lakefs_client)
# silver_repo = Repository("silver", client=lakefs_client)
# gold_repo = Repository("gold", client=lakefs_client)

### Create lakefs branch

In [6]:
bronze_repo.branch("feat-multi-table-transactions").create(source_reference="main", exist_ok=True)

Branch(repository="bronze", id="feat-multi-table-transactions")

### Write multiple tables

In [ ]:
first_table = pl.DataFrame({"foo": [1,2,3,4,5], "bar": ["delta-rs", "is", "pretty","awesome", "ey"]})
second_table = pl.DataFrame({"foo": [10,11,12,13,14], "bar": ["delta-rs", "is", "pretty","awesome", "ey"]})

first_table.write_delta(
    "lakefs://bronze/feat-multi-table-transactions/first_table", 
    storage_options=storage_options
)
second_table.write_delta(
    "lakefs://bronze/feat-multi-table-transactions/second_table", 
    storage_options=storage_options
)

### Merge as single lakefs commit into `main`

In [9]:
bronze_repo.branch("feat-multi-table-transactions").merge_into("main", squash_merge=True)

'73dcbd3db5e79b674b87634b8b4302d46b658ccaaed505c4ca1b6afc6b1c85dc'